In [11]:
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as manimation

In [12]:
#Videocodierungs-Software suchen:
print(manimation.writers.list())

['pillow', 'ffmpeg', 'ffmpeg_file', 'html']


In [24]:
#Writer für den Videoexport definieren, dabei ffmpeg nutzen:
FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Planetenanimation', artist='Maximilian Jaffke',
                comment='42')
writer = FFMpegWriter(fps=25, metadata=metadata)

In [82]:
#Daten aus der Textdatei laden, und groeße sowie maximalen x/y-Wert auslesen:
data_array = np.loadtxt('SonneMerkurUranus.txt') #Hier jeweils vor das .txt den Titel der zu visualisierenden Datei angeben 
print('Anzahl Iterationen: ',len(data_array)) 
print('Datenformat: ',data_array.shape)
print('Größter Wert: ', np.absolute(data_array).max())
max_value = np.absolute(data_array).max()

Anzahl Iterationen:  7999
Datenformat:  (7999, 6)
Größter Wert:  2933080000000.0


In [83]:
#Matplotlib-Figure definieren und für schweif und Bodys jeweils einen Plot starten, mit (noch) leeren daten
fig = plt.figure()
plots_B = []
plots_T = []
zord = [2,2,3] #lediglich um Sonne-Erde-Mond besser darzustellen, da sich Erde und Mond sonst überlagern
m_size = [4,2,1] #siehe zord
for i in range(int(data_array.shape[1]/2)):
    plots_B.append(plt.plot([], [],'o',zorder=2))

    
#Farbzyklus zurücksetzen, sodass Schweif und Körper die selbe Farbe haben    
plt.gca().set_prop_cycle(None)
    
for i in range(int(data_array.shape[1]/2)):  
    plots_T.append(plt.plot([], [], '-', lw = 1, zorder=1))

In [84]:
# Max x und y auf maxwert der Daten setzen, plus 10%: 

plt.xlim(-max_value - 0.1*max_value, max_value + 0.1*max_value)
plt.ylim(-max_value - 0.1*max_value, max_value + 0.1*max_value)


(-3226388000000.0, 3226388000000.0)

In [85]:
#Länge des Trails in Frames angeben:
trail = 500


In [86]:
#Plotten Der Daten: In jedem Durchlauf die entsprechenden Daten nehmen und mit lx.set_data an figure übergeben
with writer.saving(fig, "SMU.mp4", 200): #Hier kann vor das .mp4 der gewünschte Video Name gesetzt werden
    for i in range(len(data_array)):
        
        #Plot-Daten zurücksetzen, sodass im nächsten Frame wieder alles neu gemacht werden kann
        x_Body  = []
        y_Body  = []
        x_Trail = []
        y_Trail = []
        
        #Positionen der Körper hinzufügen
        for j in range(0, data_array.shape[1], 2):
            x_Body.append(data_array[i,j])
            y_Body.append(data_array[i,j+1])
            
        #Beim start muss der trail noch kurz sein:  
        if i -trail <0:
            begin_trail = 0
        else:
            begin_trail = i - trail
            
        #Positionen der Körper in den letzten "trail" Schritten hinzufügen:
        for j in range(0, data_array.shape[1], 2):
            x_Trail.append(data_array[begin_trail:i-1, j])
            y_Trail.append(data_array[begin_trail:i-1, j+1])
            
        #Daten Plotten:
        for j in range(0, int(data_array.shape[1]/2)):
            plots_T[j][0].set_data(x_Trail[j], y_Trail[j])
            plots_B[j][0].set_data(x_Body[j], y_Body[j])
        
        #Video-Frame grabben
        writer.grab_frame()